In [1]:
import aocd
from aocd.models import Puzzle
day = 16
year = 2020
puzzle = Puzzle(year=year, day=day)
# data = aocd.get_data(day=day, year=year)
with open('./data/input_{:02d}'.format(day), 'w') as fh:
    fh.write(puzzle.input_data)

In [2]:
data = puzzle.input_data.splitlines()
data[:10]

['departure location: 36-626 or 651-973',
 'departure station: 38-134 or 142-966',
 'departure platform: 32-465 or 489-972',
 'departure track: 40-420 or 446-973',
 'departure date: 38-724 or 738-961',
 'departure time: 30-358 or 377-971',
 'arrival location: 48-154 or 166-965',
 'arrival station: 48-669 or 675-968',
 'arrival platform: 27-255 or 276-965',
 'arrival track: 37-700 or 720-955']

In [10]:
test_data = """class: 1-3 or 5-7
row: 6-11 or 33-44
seat: 13-40 or 45-50

your ticket:
7,1,14

nearby tickets:
7,3,47
40,4,50
55,2,20
38,6,12"""

In [3]:
len(data)

267

In [54]:
def parse_conditions(line):
    field, condition = line.split(': ')
    conds = condition.split(' or ')
    res = []
    for c in conds:
        _min, _max = c.split('-')
        res.append({
            'min': int(_min),
            'max': int(_max)
        })
    return field, res

In [193]:
def valid(ticket, conditions):
    for cond in conditions.values():
        for _range in cond:
            if ticket >= _range['min'] and ticket <= _range['max']:
                return True
    return False


def parse_data(data):
    parse = 'conds'
    conditions = {}
    myticket = None
    tickets = []
    valid_tickets = []

    for line in data:
    #     print(line)
        if line.strip() == "":
            continue
        if line.startswith("your ticket"):
            parse = 'myticket'
            continue
        if line.startswith("nearby tickets"):
            parse = 'tickets'
            continue
        if parse == 'conds':
            field, conds = parse_conditions(line)
            conditions[field] = conds
        if parse == 'myticket':
            myticket = list(map(int, line.split(',')))
        if parse == 'tickets':
            tickets.append(list(map(int, line.split(','))))

    wrong = []
    for ticket in tickets:
        for num in ticket:
            if not valid(num, conditions):
                wrong.append(num)
                break
        else:
            valid_tickets.append(ticket)
            
    return myticket, valid_tickets, conditions, sum(wrong)
    
        

In [194]:
# data = test_data.splitlines()
data = puzzle.input_data.splitlines()
myticket, valid_tickets, conditions, sum_wrong = parse_data(data)
sum_wrong

30869

In [66]:
puzzle.answer_a = sum_wrong

That's the right answer!  You are one gold star closer to saving your vacation. [Continue to Part Two]


In [68]:
# Part B

In [79]:
test_data_B = """class: 0-1 or 4-19
row: 0-5 or 8-19
seat: 0-13 or 16-19

your ticket:
11,12,13

nearby tickets:
3,9,18
15,1,5
5,14,9"""

In [235]:
# myticket, valid_tickets, conditions, sum_wrong = parse_data(test_data_B.splitlines())
myticket, valid_tickets, conditions, sum_wrong = parse_data(puzzle.input_data.splitlines())

In [236]:
conditions

{'departure location': [{'min': 36, 'max': 626}, {'min': 651, 'max': 973}],
 'departure station': [{'min': 38, 'max': 134}, {'min': 142, 'max': 966}],
 'departure platform': [{'min': 32, 'max': 465}, {'min': 489, 'max': 972}],
 'departure track': [{'min': 40, 'max': 420}, {'min': 446, 'max': 973}],
 'departure date': [{'min': 38, 'max': 724}, {'min': 738, 'max': 961}],
 'departure time': [{'min': 30, 'max': 358}, {'min': 377, 'max': 971}],
 'arrival location': [{'min': 48, 'max': 154}, {'min': 166, 'max': 965}],
 'arrival station': [{'min': 48, 'max': 669}, {'min': 675, 'max': 968}],
 'arrival platform': [{'min': 27, 'max': 255}, {'min': 276, 'max': 965}],
 'arrival track': [{'min': 37, 'max': 700}, {'min': 720, 'max': 955}],
 'class': [{'min': 50, 'max': 319}, {'min': 332, 'max': 958}],
 'duration': [{'min': 35, 'max': 822}, {'min': 835, 'max': 949}],
 'price': [{'min': 40, 'max': 791}, {'min': 802, 'max': 951}],
 'route': [{'min': 42, 'max': 56}, {'min': 82, 'max': 968}],
 'row': [{'

In [237]:
N = len(conditions)
possible = {
    k: list(conditions.keys()) for k in range(N)
}

In [238]:
found = set([])
while not all([l==1 for l in map(len, possible.values())]):
    for ticket in valid_tickets:
        for i, num in enumerate(ticket):
            
            if len(possible[i]) == 1:
                field = possible[i][0]
                if field not in found:
                    found.add(field)
#                 print(possible[i][0])
                    for j in range(N):
                        if j == i:
                            continue
                        try:
                            possible[j].remove(field)
                        except ValueError:
                            pass
                continue
                
            for field in possible[i].copy():         
                for _range in conditions[field]:
                    if num >= _range['min'] and num <= _range['max']:
                        break
                else:
#                     print('remove', i, field, ticket, num)
                    possible[i].remove(field)                                
                
                

In [239]:
possible

{0: ['departure date'],
 1: ['arrival track'],
 2: ['arrival station'],
 3: ['arrival location'],
 4: ['duration'],
 5: ['class'],
 6: ['route'],
 7: ['zone'],
 8: ['departure station'],
 9: ['arrival platform'],
 10: ['departure platform'],
 11: ['departure track'],
 12: ['seat'],
 13: ['wagon'],
 14: ['price'],
 15: ['row'],
 16: ['type'],
 17: ['departure location'],
 18: ['train'],
 19: ['departure time']}

In [242]:
part_b = 1
for k, v in possible.items():
    if v[0].startswith('departure'):
        print(k, v[0], myticket[k])
        part_b *= myticket[k]

0 departure date 127
8 departure station 103
10 departure platform 97
11 departure track 179
17 departure location 101
19 departure time 191


In [243]:
puzzle.answer_b = part_b

That's the right answer!  You are one gold star closer to saving your vacation.You have completed Day 16! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
